<a href="https://colab.research.google.com/github/lukaszplust/Machine-Learning/blob/main/kmeans_lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOME TEST

There is how  random.sample works it choose without repetition

In [ ]:
#data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
#k = 3
#centroids = random.sample(data, k)
#print(centroids)

In [ ]:
#words = ["apple", "banana", "kiwi", "orangee"]

#sorted_words = sorted(words, key=lambda word: len(word))
#sorted_words = sorted(words, key=lambda word: -len(word))
#print(sorted_words)

In [27]:
import numpy as np
import random

def initialize_centroids_forgy(data, k):
    # TODO implement random initialization --> DONE

    #przekształcenie danych w liste i losowo wybierane jest k próbek z listy danych bez ich zwracania
    centroids = np.array(random.sample(list(data), k))
    return centroids


# DESCRIPTION FROM INSTRUCTION

# Algorytm inicjalizacji kmeans++ zaczyna
# się od wylosowania spośród obserwacji jednego centroida
# a następnie dodawania kolejnych pojedyńczo

# Dodawana jest zawsze ta obserwacja, której odległość
# od obecnych centroidów jest największa spośród wszystkich dostępnych obserwacji



def initialize_centroids_kmeans_pp(data, k):
    # TODO implement kmeans++ initizalization
    centroids = []
    random_centroid = random.sample(list(data), 1) # 1 randomowy centroid

    centroid = []

    # przechodze przez wszystkie indeksy w pierwszym wymiarze macierzy random_centroid
    for i in range(len(random_centroid[0])):
      # następnie dodaje odpowiadające im elementy do listy centroid
      centroid.append(random_centroid[0][i])

    centroids.append(centroid)

    print("Initial centroids:", centroids)  # Dodajmy wyświetlanie wygenerowanych centroidów

    for _ in range(k-1):
        # ustawienie wartosci minimalnych zapewnia, że w przypadku, gdy obliczona odległość między punktem danych,
        # a centroidem będzie mniejsza od bieżącej maksymalnej odległości, zostanie ona uaktualniona
        max_distance, max_index = float('-inf'), -1

        # znajduje punkt danych, który ma największą odległość od średniej pozycji wszystkich centroidów
        max_point = max(data, key=lambda point: np.linalg.norm(point - np.array(centroids).mean(axis=0)))
        centroids.append(np.array(max_point))
    return np.array(centroids)

def assign_to_cluster(data, centroid):
    # TODO find the closest cluster for each data point
    # Przypisz każdy punkt danych do najbliższego centroidu
    assignments = []
    for point in data:
        # Oblicz odległość między punktem a wszystkimi centroidami za pomocą normy euklidesowej
        distances = np.linalg.norm(centroid - point, axis=1)
        # Znajdź indeks najbliższego centroidu
        closest_centroid_index = np.argmin(distances)
        # Przypisz punkt do odpowiadającego mu klastra
        assignments.append(closest_centroid_index)
    return np.array(assignments)


def update_centroids(data, assignments):
    # TODO find new centroids based on the assignments

    # len(set(assignments)) --> zwraca liczbę unikalnych klastrów, do których zostały przypisane punkty danych.
    # każdy nowy centroid jest wyliczany jako średnia wartość punktów przypisanych do danego klastra
    return np.array([np.mean(data[np.array(assignments) == i], axis=0) for i in range(len(set(assignments)))])


def mean_intra_distance(data, assignments, centroids):
    return np.sqrt(np.sum((data - centroids[assignments, :])**2))



def k_means(data, num_centroids, kmeansplusplus= False):
    # centroids initizalization
    if kmeansplusplus:
        centroids = initialize_centroids_kmeans_pp(data, num_centroids)
    else:
        centroids = initialize_centroids_forgy(data, num_centroids)


    assignments  = assign_to_cluster(data, centroids)
    for i in range(100): # max number of iteration = 100
        print(f"Intra distance after {i} iterations: {mean_intra_distance(data, assignments, centroids)}")
        centroids = update_centroids(data, assignments)
        new_assignments = assign_to_cluster(data, centroids)
        if np.all(new_assignments == assignments): # stop if nothing changed
            break
        else:
            assignments = new_assignments

    return new_assignments, centroids, mean_intra_distance(data, new_assignments, centroids)

In [29]:
#from k_means import k_means
import pandas as pd
import numpy as np

def load_iris():
    data = pd.read_csv("iris.data", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
    #print(data)
    classes = data["class"].to_numpy()
    features = data.drop("class", axis=1).to_numpy()
    return features, classes

def evaluate(clusters, labels):
    for cluster in np.unique(clusters):
        labels_in_cluster = labels[clusters==cluster]
        print(f"Cluster: {cluster}")
        for label_type in np.unique(labels):
            print(f"Num of {label_type}: {np.sum(labels_in_cluster==label_type)}")


def clustering(kmeans_pp):
    data = load_iris()
    features, classes = data
    intra_class_variance = []
    for i in range(100):
        assignments, centroids, error = k_means(features, 3, kmeans_pp)
        evaluate(assignments, classes)
        intra_class_variance.append(error)
    print(f"Mean intra-class variance: {np.mean(intra_class_variance)}")

if __name__=="__main__":
    clustering(kmeans_pp = True)
    #clustering(kmeans_pp = False)

Initial centroids: [[6.4, 3.2, 5.3, 2.3]]
Intra distance after 0 iterations: 15.362291495737216
Intra distance after 1 iterations: 10.308319681757139
Intra distance after 2 iterations: 9.682201656156298
Intra distance after 3 iterations: 9.235722035750864
Intra distance after 4 iterations: 8.941665638754039
Intra distance after 5 iterations: 8.90429910996805
Cluster: 0
Num of Iris-setosa: 0
Num of Iris-versicolor: 48
Num of Iris-virginica: 14
Cluster: 1
Num of Iris-setosa: 50
Num of Iris-versicolor: 0
Num of Iris-virginica: 0
Cluster: 2
Num of Iris-setosa: 0
Num of Iris-versicolor: 2
Num of Iris-virginica: 36
Initial centroids: [[6.9, 3.2, 5.7, 2.3]]
Intra distance after 0 iterations: 17.956614380222128
Intra distance after 1 iterations: 10.43738890924346
Intra distance after 2 iterations: 9.81755024404539
Intra distance after 3 iterations: 9.294266283035522
Intra distance after 4 iterations: 8.941665638754039
Intra distance after 5 iterations: 8.90429910996805
Cluster: 0
Num of Iris-s

Odległość wewnątrzgrupowa maleje wraz z kolejnymi iteracjami, co sugeruje, że algorytm zmierza do znalezienia optymalnego położenia centroidów